In [4]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from __future__ import division

import os
import numpy as np
import time
import signal
import scipy.io as scio
from scipy.sparse import coo_matrix
from MDPD import *
from MDPD.readers import *
import matplotlib.pyplot as plt
import matplotlib

----
## Read Data

In [5]:
folder = '/media/vzhao/Data/crowdsourcing_datasets/bird'
train = Crowd_Sourcing_Readers.read_data(os.path.join(folder, 'bluebird_crowd.txt'))
label = Crowd_Sourcing_Readers.read_label(os.path.join(folder, 'bluebird_truth.txt'))

In [7]:
model = MDPD.MDPD()
model.fit(train, ncomp=2, verbose=True)
model.score(train, label)

Traceback (most recent call last):
  File "/usr/lib/python2.7/logging/__init__.py", line 861, in emit
    msg = self.format(record)
  File "/usr/lib/python2.7/logging/__init__.py", line 734, in format
    return fmt.format(record)
  File "/usr/lib/python2.7/logging/__init__.py", line 465, in format
    record.message = record.getMessage()
  File "/usr/lib/python2.7/logging/__init__.py", line 329, in getMessage
    msg = msg % self.args
TypeError: %d format: a number is required, not NoneType
Logged from file MDPD.py, line 226
2018-03-28 15:18:22,246 : INFO : iteration 0; log-likelihood -17.535398;
2018-03-28 15:18:22,248 : INFO : iteration 1; log-likelihood -17.493472;
2018-03-28 15:18:22,251 : INFO : iteration 2; log-likelihood -17.485623;
2018-03-28 15:18:22,253 : INFO : iteration 3; log-likelihood -17.483747;
2018-03-28 15:18:22,255 : INFO : iteration 4; log-likelihood -17.482978;
2018-03-28 15:18:22,258 : INFO : iteration 5; log-likelihood -17.482689;
2018-03-28 15:18:22,260 : INFO

0.8981481481481481